# Tutorial 1: Basic SMPL-X

Author: [Hejia Chen](http://harryxd2018.github.io), BUAA

In this notebook, we will explore the basic implementation of SMPL-X model.

## Definitions

Briefly, SMPL, FLAME and MANO are statistical model for human body, face and hands. Based on which, the SMPL+H and SMPL-X are proposed, as:

- SMPL+H: SMPL with articulated hands,
- SMPL-X: SMPL with hands and facial expression.

We strongly recommend you to read the [SMPL](https://smpl.is.tue.mpg.de/), [FLAME](http://flame.is.tue.mpg.de/) website and papers for more details.

## Requirements
To run this tutorial, you need to install the following packages:
```bash
pip install torch, smplx
```
Note that the smplx package is not necessary for this notebook, but it is required for the following tutorials.

## Download the models
Please download the SMPL-family models in `.pkl` files from official websites ([SMPL](https://smpl.is.tue.mpg.de/), [SMPL+H](http://mano.is.tue.mpg.de/), [SMPL-X](https://smplx.is.tue.mpg.de/)) after registering. Then, put the models under the `models` folder, as:

In [1]:
! tree ./models /F

卷 Data 的文件夹 PATH 列表
卷序列号为 7648-753B
D:\CODES\SMPL\MODELS
├─mano
│      MANO_LEFT.pkl
│      MANO_RIGHT.pkl
│      
├─smpl
│      SMPL_FEMALE.pkl
│      SMPL_MALE.pkl
│      SMPL_NEUTRAL.pkl
│      
├─smplh
│      SMPLH_female.pkl
│      SMPLH_male.pkl
│      
└─smplx
        SMPLX_FEMALE.pkl
        SMPLX_MALE.pkl
        SMPLX_NEUTRAL.pkl
        


## Import the packages

We will need:
- `torch`: to conduct matrix operations,
- `pickle`: to load the SMPL-family models.

In [2]:
import torch
import pickle

## Load the model

Here we choose `smplx/SMPLX_NEUTRAL.pkl` as an example. The model is loaded as a dictionary, with the following keys:

In [3]:
with open('./models/smplx/SMPLX_NEUTRAL.pkl', 'rb') as f:
    smplx_neutral = pickle.load(f, encoding='latin1')
print(smplx_neutral.keys())

dict_keys(['dynamic_lmk_bary_coords', 'hands_componentsl', 'ft', 'lmk_faces_idx', 'f', 'J_regressor', 'hands_componentsr', 'kintree_table', 'hands_coeffsr', 'joint2num', 'hands_meanl', 'lmk_bary_coords', 'weights', 'posedirs', 'dynamic_lmk_faces_idx', 'part2num', 'vt', 'hands_meanr', 'hands_coeffsl', 'v_template', 'shapedirs'])


The parameter is stored in `np.ndarray` format, so we need to convert it to `torch.Tensor` for further operations.

In [4]:
import numpy as np
for key, value in smplx_neutral.items():
    if isinstance(value, np.ndarray):
        if key in ['joint2num', 'part2num']:
            continue
        elif key in ['ft', 'f']:
            value = value.astype(np.int64)
        smplx_neutral[key] = torch.from_numpy(value)

For convenience, we define a function to load the model, as:
```python
from utils import load_pkl
smplx_neutral = load_pkl('./models/smplx/SMPLX_NEUTRAL.pkl', to_torch=True)
```

## Visualize the model

To visualize the model, we save the model as `.obj` file, which is consist of a list of vertices and faces.

In [5]:
smplx_neutral['v_template'].shape, smplx_neutral['f'].shape

(torch.Size([10475, 3]), torch.Size([20908, 3]))

SMPL-X mesh contains 10475 vertices and 20950 faces. Now we save the model as `.obj` file.

In [6]:
with open('obj/smplx_neutral.obj', 'w') as f:
    for v in smplx_neutral['v_template']:
        f.write(f'v {v[0]} {v[1]} {v[2]}\n')
    for face in smplx_neutral['f']:
        f.write(f'f {face[0]+1} {face[1]+1} {face[2]+1}\n')

Now we can visualize the model in [MeshLab](https://www.meshlab.net/).

For convenience, we define a function to save the model as `.obj` file, as:
```python
from utils import write_obj
write_obj(smplx_neutral['v_template'], smplx_neutral['f'], 'obj/smplx_neutral.obj')
```
and its reverse function:
```python
from utils import load_obj
vertices, faces = load_obj(file_name='obj/smplx_neutral.obj')
```